<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/main/Exercises/day-6/Data_Cleaning_Feature_Engineering/FE_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Cleaning and Feature Engineering Pipeline in Google Colab
Build a reusable data pipeline that cleans and engineers features on a real-world dataset. This exercise uses the Titanic passenger dataset.

**Objectives**
- Load and inspect the Titanic dataset.

- Identify and handle missing, inconsistent, and duplicate values.

- Create new informative features.

- Modularize your pipeline into functions for cleaning and feature engineering.

- Save the final prepared DataFrame for modeling.

In [9]:
# Step 1: Install and Import Libraries

!pip install pandas numpy -q

import pandas as pd
import numpy as np


In [10]:
# Step 2: Load the Titanic Dataset

url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
# Step 3: Data Cleaning
# 1. Detect Missing Values and Data Types

print(df.info())
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int6

In [12]:
# 2. Drop Irrelevant Columns

df.drop(columns=["PassengerId","Ticket","Cabin"], inplace=True)


In [13]:
# Impute Missing Values

# Age: fill with median
df["Age"].fillna(df["Age"].median(), inplace=True)
# Embarked: fill with mode
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)


/tmp/ipython-input-204792369.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)
/tmp/ipython-input-204792369.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [14]:
# Handle Inconsistencies & Duplicates

# Ensure no negative fares
df = df[df["Fare"] >= 0]
# Drop duplicate rows
df.drop_duplicates(inplace=True)


In [15]:
# Step 4: Feature Engineering
# 1.Title Extraction from Name

df["Title"] = df["Name"].str.extract(r",\s*([^\.]+)\.")
df["Title"] = df["Title"].replace(["Lady","Countess","Capt","Col","Don","Dr","Major","Rev","Sir","Jonkheer","Dona"], "Rare")


In [16]:
# Family Size & IsAlone Flag

df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df["IsAlone"] = np.where(df["FamilySize"] == 1, 1, 0)


In [17]:
# One-Hot Encoding Categorical Features

df_final = pd.get_dummies(df, columns=["Sex","Embarked","Title"], drop_first=True)


In [23]:
def clean_data(df):
    df = df.drop(columns=["PassengerId","Ticket","Cabin"], errors="ignore")
    df = df.copy()  # ensure we work on a fresh copy
    # Fill missing Age and Embarked by direct assignment
    median_age = df["Age"].median()
    df["Age"] = df["Age"].fillna(median_age)
    mode_embarked = df["Embarked"].mode()[0]
    df["Embarked"] = df["Embarked"].fillna(mode_embarked)
    df = df[df["Fare"] >= 0].drop_duplicates()
    return df

def engineer_features(df):
    df = df.copy()
    # Extract and binarize Title
    df["Title"] = df["Name"].str.extract(r",\s*([^\.]+)\.")
    rare_titles = ["Lady","Countess","Capt","Col","Don","Dr","Major",
                   "Rev","Sir","Jonkheer","Dona"]
    df["Title"] = df["Title"].replace(rare_titles, "Rare")
    # Family size and alone flag
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"] = np.where(df["FamilySize"] == 1, 1, 0)
    # Binning
    df["FareBin"] = pd.qcut(df["Fare"], 4, labels=False)
    df["AgeBin"]  = pd.cut(df["Age"], bins=[0,12,20,40,60,100], labels=False)
    # One-hot encoding via direct assignment
    df = pd.get_dummies(df, columns=["Sex","Embarked","Title"], drop_first=True)
    return df

# Run pipeline
df_clean = clean_data(df)
df_prepared = engineer_features(df_clean)
df_prepared.head()


,Survived,Pclass,Name,Age,SibSp,Parch,Fare,FamilySize,IsAlone,FareBin,...,Embarked_Q,Embarked_S,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rare,Title_the Countess
0,0,3,"Braund, Mr. Owen Harris",22.0,1,0,7.2500,2,0,0,...,False,True,False,False,False,True,False,False,False,False
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,71.2833,2,0,3,...,False,False,False,False,False,False,True,False,False,False
2,1,3,"Heikkinen, Miss. Laina",26.0,0,0,7.9250,1,1,1,...,False,True,True,False,False,False,False,False,False,False
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,53.1000,2,0,3,...,False,True,False,False,False,False,True,False,False,False
4,0,3,"Allen, Mr. William Henry",35.0,0,0,8.0500,1,1,1,...,False,True,False,False,False,True,False,False,False,False


In [20]:
# Step 6: Save the Prepared Dataset
df_prepared.to_csv("titanic_prepared.csv", index=False)


## Reflection Questions
- Which cleaning steps had the largest impact on data consistency?

- How did the new features (Title, FamilySize, IsAlone, bins) improve the dataset’s expressive power?

- How would you adapt this pipeline for a different tabular dataset?